In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import ipyvolume as ipv
import pathlib, glob
from scipy.stats import norm
import time

In [3]:
import sys
sys.path.append('../src/')
from molecular_handling import particle
from forward_modeling import project_volume, slice_volume, rotate_volume, take_slice, project_volume_bis, backprojection, add_slice
from scipy.interpolate import RegularGridInterpolator, griddata

In [4]:
data_directory = pathlib.Path('..') / 'data'  # directory where the data is
output_dir = data_directory / 'test'
if output_dir.exists():
    print(f'Data directory {output_dir} exists: ')
    for x in list(output_dir.glob('*')):
        print(f'>>> {str(x)}') 

Data directory ../data/test exists: 
>>> ../data/test/particle_hires_xyz.npy
>>> ../data/test/particle_oddres_meta.npy
>>> ../data/test/particle_hires_map.npy
>>> ../data/test/.DS_Store
>>> ../data/test/particle_meta.npy
>>> ../data/test/het_particle_meta.npy
>>> ../data/test/particle_hires_data.npy
>>> ../data/test/particle_hires_meta.npy
>>> ../data/test/particle_map.npy
>>> ../data/test/particle_oddres_data.npy
>>> ../data/test/particle_xyz.npy
>>> ../data/test/particle_oddres_map.npy
>>> ../data/test/het_particle_data.npy
>>> ../data/test/particle_oddres_xyz.npy
>>> ../data/test/het_particle_map.npy
>>> ../data/test/particle_data.npy


In [5]:
keyword='het_particle'
dataset     = np.load(f'{output_dir}/{keyword}_data.npy')
metadataset = np.load(f'{output_dir}/{keyword}_meta.npy')
volumes      = np.load(f'{output_dir}/{keyword}_map.npy')

In [6]:
ipv.quickvolshow(volumes[0], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

/Users/michaelresplandy/miniconda3/envs/gmm-cryoem/lib/python3.6/site-packages/ipyvolume/serialize.py:92: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0] ** 2 + gradient[1] ** 2 + gradient[2] ** 2)


In [7]:
ipv.quickvolshow(volumes[1], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

In [8]:
ipv.quickvolshow(volumes[2], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

# Reconstructing volumes with variable $z_i$  and orientations known 

In [9]:
start = time.time()
vols_reconstructed = np.zeros(volumes.shape)
for i in range(volumes.shape[0]):
    index_i = (metadataset[:,5] == i)
    vols_reconstructed[i] = backprojection(dataset[index_i], metadataset[index_i][:,0:3])
end = time.time()
print(f"time: {end-start}, sum of voxels: {np.sum(vols_reconstructed)}")
print(f"sum of pixels for real volume: {np.sum(volumes)}")

time: 0.9624619483947754, sum of voxels: 4230.554475289484
sum of pixels for real volume: 4230.958957810194


In [10]:
ipv.quickvolshow(vols_reconstructed[0], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

In [11]:
ipv.quickvolshow(vols_reconstructed[1], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

In [12]:
ipv.quickvolshow(vols_reconstructed[2], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

# Reconstructing volumes with hidden variable $z_i$ unknown but orientations known

In [13]:
def proba(projected_vol, image, scale=10):
    """
    This function calculates a probability score to tell how close two images are.
    """
    return np.prod((1-norm.cdf(np.abs(projected_vol-image), loc=0, scale=scale))*2)

def het_reconstruct(dataset, metadataset, vols, heterogeneity = False):
    """
    This function reconstructs the models at step t+1 given the models at step t
    To long, mainly because of the projection operator
    """    
    
    counts = np.zeros(vols.shape, dtype=complex)
    next_models = np.zeros(vols.shape, dtype=complex)

    for i in range(len(dataset)):
        images_i = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(dataset[i])))
        rot = R.from_rotvec(-metadataset[i,0:3])                            
        #Adding the i-th slice in the "rot"-rotated plane weighed by the similarity score 
        if heterogeneity:
            prob = np.zeros(vols.shape[0])
            for k in range(vols.shape[0]):
                projected_vol = project_volume(vols[k], rot)
                prob[k] = proba(projected_vol, dataset[i])
                next_models[k], counts[k] = add_slice(next_models[k], counts[k], images_i, rot, prob = prob[k]) 
                #we add the image, weighed by the probabity
                #to construct the models at the next iteration
            #k_ = np.argmax(prob)
            #next_models[k_], counts[k_] = add_slice(next_models[k_], counts[k_], images_i, rot) 
        else:
            projected_vol = project_volume(vols, rot)
            prob = proba(projected_vol, dataset[i])                                      
            #probs+= prob
            next_models, counts = add_slice(next_models, counts, images_i, rot, prob = prob)

    
    #Dividing by count to scale the solution/ taking the inverse fourier transform
    if (heterogeneity):
        for k in range(next_models.shape[0]):
            counts[k][counts[k] == 0] = 1
            next_models[k] = next_models[k]/counts[k]
            next_models[k] = np.real(np.fft.fftshift(np.fft.ifftn(np.fft.fftshift(next_models[k]))))
                                                        
    else:
        counts[counts == 0] = 1
        next_models = next_models/counts
        next_models = np.real(np.fft.fftshift(np.fft.ifftn(np.fft.fftshift(next_models))))
        
    return next_models 

In [14]:
het_vols_reconstructed = het_reconstruct(dataset, metadataset, volumes, heterogeneity = True)

In [15]:
ipv.quickvolshow(het_vols_reconstructed[0], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

/Users/michaelresplandy/miniconda3/envs/gmm-cryoem/lib/python3.6/site-packages/traitlets/traitlets.py:1985: ComplexWarning: Casting complex values to real discards the imaginary part
  value = float(value)
/Users/michaelresplandy/miniconda3/envs/gmm-cryoem/lib/python3.6/site-packages/ipyvolume/serialize.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  subdata[..., 3] = (Im * 255).astype(np.uint8)
/Users/michaelresplandy/miniconda3/envs/gmm-cryoem/lib/python3.6/site-packages/ipyvolume/serialize.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  subdata[..., i] = ((gradient[i][zindex] / 2.0 + 0.5) * 255).astype(np.uint8)


In [16]:
ipv.quickvolshow(het_vols_reconstructed[1], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

In [17]:
ipv.quickvolshow(het_vols_reconstructed[2], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)